<a href="https://colab.research.google.com/github/turab45/Audd-multi-modality-for-urdu-audio/blob/master/Audd_multi_modality_for_urdu_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!unzip '/content/drive/MyDrive/dataset_old.zip'
#!git clone https://github.com/Nikunj1729/free-spoken-gujarati-digit-dataset

In [ ]:
# PROCESSING
# Feature extraction methods

def CalculateZeroCrossingRate(file_location):
    y, sr = librosa.load(file_location)
    zCross = librosa.feature.zero_crossing_rate(y=y)
    dim = (32, 32)
    resized = cv2.resize(zCross, dim, interpolation = cv2.INTER_AREA)
    return resized

def CalculateMelSpectrogram(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr)
    melSpec_dB = librosa.power_to_db(melSpec)
    dim = (32, 32)
    resized = cv2.resize(melSpec_dB, dim, interpolation = cv2.INTER_AREA)
    return resized


def CalculateMFCC(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.mfcc(y=y, sr=sr)
    dim = (32, 32)
    resized = cv2.resize(melSpec, dim, interpolation = cv2.INTER_AREA)
    return resized



In [ ]:
cd /content

/content


In [ ]:
'''for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue 
      print(t.split("WA")[1].split(".")[0])
      #print(t.split("D")[1][0])
      break
    break 
  break '''

['7', '8', '1', '6', '3', '2', '5', '4', '0', '9']
7
6614


In [ ]:
# PROCESSING

files_path = '/content/drive/MyDrive/audd saved files'
if not os.path.exists("processed_data"):
  os.mkdir("processed_data")
x=[]
y=[]
Total=23500
count=0

j = '/content/dataset'
print(os.listdir(j))
for k in os.listdir(j):
  print(k)
  if ".DS_" in k:
    continue

  for t in os.listdir(j+"/"+k):
    #print("path = ", os.listdir(j+"/"+k))
    if ".DS_" in k:
      continue
    print(count,t)
    x.append(CalculateZeroCrossingRate((j+"/"+k+"/"+t))) #CalculateZeroCrossingRate(), CalculateMelSpectrogram(), CalculateMFCC() 
    y.append(int(k))   
    count+=1
  if count%300==0:
    print(Total-count)
  
x=np.array(x)
y=np.array(y)
np.save(files_path+"/X_zero_crossing_rate",x) # _melspectrogram, zero_crossing_rate, mfcc
np.save(files_path+"/y_zero_crossing_rate",y) 

In [ ]:
#load files 
files_path = '/content/drive/MyDrive/audd saved files'

x_melspectrogram = np.load(files_path+"/X_mfcc.npy", allow_pickle=True)
y_melspectrogram = np.load(files_path+"/y_mfcc.npy", allow_pickle=True)

In [ ]:
#Split data into train and test

from sklearn.model_selection import train_test_split
x = x_melspectrogram
y = y_melspectrogram

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_mfcc_train",x_train)
np.save(files_path+"/y_mfcc_train",y_train)
np.save(files_path+"/x_mfcc_test",x_test)
np.save(files_path+"/y_mfcc_test",y_test)

In [ ]:
# Load data
x_train = np.load(files_path+"/x_mfcc_train.npy", allow_pickle=True)
x_test = np.load(files_path+"/x_mfcc_test.npy",allow_pickle=True) 
y_train = np.load(files_path+"/x_mfcc_train.npy", allow_pickle=True)
y_test = np.load(files_path+"/y_mfcc_test.npy",allow_pickle=True)

In [ ]:
print(np.unique(y_melspectrogram))

[0 1 2 3 4 5 6 7 8 9]


In [ ]:
x_train =x_train.reshape((x_train.shape[0],32,32,1))
x_test =x_test.reshape((x_test.shape[0],32,32,1))

## Train Model

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
def GetCNN():
  model = models.Sequential()
  
  model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())
  
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Dropout(0.1))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))# fully connected
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.1))
  
  model.add(layers.Dense(10,activation='softmax', use_bias=True))
  return model 

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
train_y = label_as_binary.fit_transform(y_train)
test_y = label_as_binary.fit_transform(y_test)

In [ ]:
save_path = '/content/drive/MyDrive/audd saved files/models'


from tensorflow.keras.callbacks import ModelCheckpoint
cnn_model = GetCNN()
print(cnn_model.summary())
cnn_model.compile(loss  = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ["accuracy"],
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
callbacks = ModelCheckpoint(save_path+"/melspectrogram_model_.h5", monitor='val_accuracy',
                                    mode='max',
                                    save_best_only=True,
                                    verbose=1)

print("X train shape = ",x_train.shape)
print("y train shape = ",train_y.shape)

cnn_model.fit(x_train, train_y, batch_size=64, epochs=150, verbose=1, validation_split=0.1, callbacks=[callbacks])
#import keras
#cnn_model = keras.models.load_model(save_path+"/melspectrogram_model_.h5")
#print("Test Accuracy is ", cnn_model.evaluate(x_test,test_y)[1])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 30, 30, 64)        640       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 15, 15, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 13, 13, 64)        36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                      

ValueError: ignored